In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from matplotlib.pyplot import imread,imshow
import matplotlib.pyplot as plt
from skimage.transform import resize
from sklearn.utils import shuffle
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import glob
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist

# Preparing Data

In [ ]:
train_df = pd.read_csv("../input/digit-recognizer/train.csv")
test_df  = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
xtr = np.array(train_df.drop(columns=['label']))
ytr = np.array(train_df['label'])

xte = np.array(test_df)

In [ ]:
xtr = xtr.reshape((xtr.shape[0],28,28,1))
xte = xte.reshape((xte.shape[0], 28,28,1))

xtr = xtr.astype('float32')
xte = xte.astype('float32')

xtr = xtr/ 255
xte =xte/ 255

In [ ]:
ytr = to_categorical(ytr,10)

# Creating Model

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(7, 7), strides=(1, 1), activation='relu',input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(BatchNormalization(momentum=0.9, epsilon=1e-5, gamma_initializer="uniform"))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics='accuracy')
model.summary()

In [ ]:
history = model.fit(xtr,ytr,batch_size=100,epochs=20,validation_split = 0.1)

# Sample demonstration

In [ ]:
rows = 4
columns = 4

fig = plt.figure(figsize=(16,16))

random_images = []

for _ in range(rows*columns):
    random_index = np.random.randint(0,xte.shape[0])
    
    random_images.append(xte[random_index])
    
random_images = np.array(random_images)

predictions = np.argmax(model.predict(random_images),1).astype('int32').reshape(rows*columns,)

for i in range(1,rows*columns+1):
    ax = plt.subplot(rows, columns, i)
    plt.imshow(random_images[i-1])
    predicted_str = "Predicted: " + str(predictions[i-1])
    ax.set_title(predicted_str)

fig.show()

# Submitting Results

In [ ]:
predictions = model.predict(xte)

In [ ]:
pred = np.argmax(predictions,axis=1)

In [ ]:
submission_df = pd.DataFrame()

imageId = [i for i in range(1,pred.shape[0]+1)]
submission_df['ImageId'] = imageId
submission_df['Label'] = pred

submission_df.to_csv("submission.csv",index=False)